# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686562


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:09,  2.50s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:48,  1.79s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:33,  1.30s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:34,  1.38s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  6.01it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.26it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.51it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  6.76it/s]

T_atom                                0.001682
jammiedodgers                         0.006310
JonKerly                              0.001374
not-logged-in-a9247261537fa1da33fe    0.043618
cschwefl                              0.001927
Dynamatt                              0.001905
djswanso                              0.001699
mtajnai                               0.001982
bldelacr                              0.001665
nsyee                                 0.001443
not-logged-in-6fdf43e5a9997ae450a2    0.002639
clars915                              0.001672
raisamaduro                           0.004395
not-logged-in-5f30b2bb9b7f9d49da41    1.265213
Saharisunshine                        0.001803
acapirala                             0.001687
aidenr                                0.001515
not-logged-in-bf4f57cdbd589b54f822    0.008195
Lavadude                              1.185813
jmacd297                              0.004491
mcooke27                              0.001625
FR4N1O       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())